In [1]:
import numpy as np
import pickle
import os
from pinocchio.libpinocchio_pywrap import SE3
from database import *
from croc_lib import *
import crocoddyl
import matplotlib.pyplot as plt
from crocoddyl import loadTalos
%load_ext autoreload
%autoreload 2

In [2]:
from memmo_utils import Visual
viz = Visual(rate=25)

### Previous Version

### Current Version 

#### Read from the dataset

In [3]:
#FILE_CAT = 'talos_moveEffector_flat'
FILE_CAT = 'talos_circle'
#FILE_CAT = 'talos_circle_oriented'
ROOT = '/media/teguh/Data/MEMMO Dataset/' + FILE_CAT + '/'
ROOT_PROCESSED = '/media/teguh/Data/MEMMO Dataset/processed_data/' + FILE_CAT + '/'

file_names = os.listdir(ROOT)

In [4]:
np.set_printoptions(precision=4,suppress=True)

In [8]:
viz.set_rate(1000)

In [9]:
for j in range(10):
    i = np.random.randint(len(file_names))
    filename = file_names[i]
    try:
        traj, vel_traj, u_traj, T_lefts, T_rights, data_phases, delta_t, stat, \
        res = extract_from_dataset(ROOT+filename, FILE_CAT, False, 1, version='v1.0')
        break
    except:
        print 'Fail'
        continue


viz.vis_traj(traj)

Result constructor called without contactSequence object, phase_interval member not initialized
Obtain a valid trajectory file!
(7051, 39)


#### Plot the root or COM trajectories

#### Store into the format of crocoddyl

In [ ]:
phases = store_for_crocoddyl(timesteps, lfs, rfs, trajs, vel_trajs, croc_timestep)
pickle.dump(phases,open('data/croc_prob_files/' + file_name ,'wb'))

In [10]:
phases = store_for_crocoddyl(traj, vel_traj, u_traj, T_lefts, T_rights, contact_act_left, contact_act_right, data_phases, delta_t)
pickle.dump(phases,open('data_teguh_3.txt','wb'))

NameError: name 'contact_act_left' is not defined

### Run Crocoddyl 

In [16]:
ROBOT = loadTalos('/opt/openrobots/share/example-robot-data')
problem, xs, us, ts = define_croc_problem(ROBOT,'data_teguh_3.txt', num_phases=3, is_warmstart=True)

solver = solve_problem(ROBOT,problem,xs, us,maxiter = 50, STOP_THRESHOLD=1e-03, recalc_u=True)
traj = np.array(solver.xs)[:,0:39]
viz.vis_traj(traj)

Building the set of action models and warmstart from Memmo
phase 0:
  nodes: 39
  contacts:
    suppport: ['right_sole_link', 'left_sole_link']
    swing: []
phase 1:
  nodes: 35
  contacts:
    suppport: ['left_sole_link']
    swing: ['right_sole_link']
phase 2:
  nodes: 37
  contacts:
    suppport: ['right_sole_link', 'left_sole_link']
    swing: []

*** SOLVE ***
iter 	 cost 	      stop 	    grad 	  xreg 	      ureg 	 step 	 feas
   0  3.60707e+03  4.58755e+00  1.31552e+03  1.00000e-09  1.00000e-09   1.0000     1
   1  2.01116e+03  2.33381e+01  3.76573e+03  1.00000e-09  1.00000e-09   1.0000     1
   2  9.66521e+02  1.20531e+02  2.21085e+03  1.00000e-09  1.00000e-09   1.0000     1
   3  7.70344e+02  3.65451e+01  9.55600e+02  1.00000e-09  1.00000e-09   1.0000     1
   4  4.99688e+02  1.21254e+00  7.52229e+02  1.00000e-09  1.00000e-09   1.0000     1
   5  3.19511e+02  7.71690e+00  3.78298e+02  1.00000e-09  1.00000e-09   1.0000     1
   6  2.42841e+02  4.28191e+00  1.96047e+02  1.00000e